# Get data and packages

In [1]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
from pyomo.environ import *
import pyomo.opt as pyopt

from data_cleaning import (blockgroup_pop_dict, bg_ces_dict,
                           dist_to_site_df, dist_to_site_dict,
                           county_prop_ealp_dict, site_kw_occ_dict,
                           site_sqft_dict, site_cost_dict)

# Build base model

In [2]:
import deployment_models
import importlib
importlib.reload(deployment_models)

model_base, bg_with_no_hub = deployment_models.build_base_model(site_cost_dict, site_kw_occ_dict, blockgroup_pop_dict, bg_ces_dict, dist_to_site_df)

# Solving different models
## Ideas
1. p_median
2. demand_maximization
3. p_median with minimum total population
4. p_median with minimum total EJ population
5. demand_maximization with minimum total EJ population
6. demand_maximization with minimum blockgroup EJ population
7. ...

# Basic p median model

In [9]:
importlib.reload(deployment_models)

max_cost = 20000000
min_service_fraction = 0.1

model_dict = dict()
######### Basic p median #############
# p median objective and constraint for total cost and to meet defined proportion of total population
model_key = "p_median"
model_dict[model_key] = model_base.clone()
model_dict[model_key] = deployment_models.constrain_total_cost(model_dict[model_key],max_cost)
model_dict[model_key] = deployment_models.constrain_min_total_pop(model_dict[model_key],min_service_fraction)
model_dict[model_key] = deployment_models.add_p_median_objective(model_dict[model_key])
results = SolverFactory('gurobi').solve(model_dict[model_key])
print(results)
var_hub_yn, var_prop_served = deployment_models.get_vars(model_dict[model_key])


Problem: 
- Name: x24130
  Lower bound: 30968.02078247392
  Upper bound: 30970.818264228175
  Number of objectives: 1
  Number of constraints: 27066
  Number of variables: 24130
  Number of binary variables: 558
  Number of integer variables: 558
  Number of continuous variables: 23572
  Number of nonzeros: 212698
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 1.6042029857635498
  Error rc: 0
  Time: 1.8213860988616943
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [12]:
print("Fraction of hubs built: ", np.sum(var_hub_yn)/len(var_hub_yn))

Fraction of hubs built:  0    0.100358
dtype: float64


# p-median to meet a minimum number of EJ population

In [15]:
######### Basic p median + EJ #############
# p_median model plus meet a certain propotion of the total EJ population
min_prop_ej = 0.2
ej_cutoff = 0.75

model_key = "p_median_total_EJ"
model_dict[model_key] = model_base.clone()
model_dict[model_key] = deployment_models.constrain_total_cost(model_dict[model_key],max_cost)
model_dict[model_key] = deployment_models.constrain_min_total_pop(model_dict[model_key],min_service_fraction)
model_dict[model_key] = deployment_models.add_p_median_objective(model_dict[model_key])
model_dict[model_key] = deployment_models.prioritize_CES(model_dict[model_key],min_prop_ej,ej_cutoff)

SolverFactory('gurobi').solve(model_dict[model_key])
print(results)
var_hub_yn, var_prop_served = deployment_models.get_vars(model_dict[model_key])
print("Fraction of hubs built: ", np.sum(var_hub_yn)/len(var_hub_yn))


Problem: 
- Name: x24130
  Lower bound: 30968.02078247392
  Upper bound: 30970.818264228175
  Number of objectives: 1
  Number of constraints: 27066
  Number of variables: 24130
  Number of binary variables: 558
  Number of integer variables: 558
  Number of continuous variables: 23572
  Number of nonzeros: 212698
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 1.6042029857635498
  Error rc: 0
  Time: 1.8213860988616943
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Fraction of hubs built:  0    0.094982
dtype: float64


# p-median to meet a certain fraction for EACH blockgroup
As expected, with the current model, this is infeasible for practically all values since hubs are all or nothing

In [21]:
importlib.reload(deployment_models)
min_blockgroup_service_fraction = 0.01

model_key = "p_median_bg_prop"
model_dict[model_key] = model_base.clone()
model_dict[model_key] = deployment_models.constrain_total_cost(model_dict[model_key],max_cost)
model_dict[model_key] = deployment_models.constrain_min_blockgroup_pop(model_dict[model_key],min_blockgroup_service_fraction)
model_dict[model_key] = deployment_models.add_p_median_objective(model_dict[model_key])
model_dict[model_key] = deployment_models.prioritize_CES(model_dict[model_key],min_prop_ej,ej_cutoff)

SolverFactory('gurobi').solve(model_dict[model_key])
print(results)
var_hub_yn, var_prop_served = deployment_models.get_vars(model_dict[model_key])
print("Fraction of hubs built: ", np.sum(var_hub_yn)/len(var_hub_yn))

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Model was proven to be infeasible.

Problem: 
- Name: x24130
  Lower bound: 30968.02078247392
  Upper bound: 30970.818264228175
  Number of objectives: 1
  Number of constraints: 27066
  Number of variables: 24130
  Number of binary variables: 558
  Number of integer variables: 558
  Number of continuous variables: 23572
  Number of nonzeros: 212698
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 1.6042029857635498
  Error rc: 0
  Time: 1.8213860988616943
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Fraction of hubs built:  0    1.0
dtype: float64


# Basic demand maximization simple model

In [22]:
######### Basic demand maximization #############
# Demand maximization objective and constraint for total cost and to meet defined proportion of total population
model_key = "demand_max"
model_dict[model_key] = model_base.clone()
model_dict[model_key] = deployment_models.constrain_total_cost(model_dict[model_key],max_cost)
model_dict[model_key] = deployment_models.add_demand_maximization_objective(model_dict[model_key])
results = SolverFactory('gurobi').solve(model_dict[model_key])
print(results)
var_hub_yn, var_prop_served = deployment_models.get_vars(model_dict[model_key])


Problem: 
- Name: x24130
  Lower bound: 136564.9141521878
  Upper bound: 136564.9141521878
  Number of objectives: 1
  Number of constraints: 27065
  Number of variables: 24130
  Number of binary variables: 558
  Number of integer variables: 558
  Number of continuous variables: 23572
  Number of nonzeros: 189127
  Sense: maximize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 46.61024284362793
  Error rc: 0
  Time: 46.812371015548706
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



# Multi-objective models
Meta-code:
For a given total amount of money...
1. Find the maximum number of people that can be met with that money using demand_maximization
2. Set the min_service_fraction equal to some fraction of the maximum number of people (say 90%)
3. Find the p-median model result for that value of min_service_fraction


In [30]:
min_service_fraction_multiplier = 0.75
total_pop = sum([model_base.param_bg_pop[bg] for bg in model_base.idx_bgs])

min_service_fraction_multi_objective = min_service_fraction_multiplier*value(model_dict["demand_max"].max_coverage)/total_pop

model_key = "p_median_multi_objective"
model_dict[model_key] = model_base.clone()
model_dict[model_key] = deployment_models.constrain_total_cost(model_dict[model_key],max_cost)
model_dict[model_key] = deployment_models.constrain_min_total_pop(model_dict[model_key],min_service_fraction_multi_objective)
model_dict[model_key] = deployment_models.add_p_median_objective(model_dict[model_key])
results = SolverFactory('gurobi').solve(model_dict[model_key])
print(results)
var_hub_yn, var_prop_served = deployment_models.get_vars(model_dict[model_key])



Problem: 
- Name: x24130
  Lower bound: 25587.173156070534
  Upper bound: 25588.525833607957
  Number of objectives: 1
  Number of constraints: 27066
  Number of variables: 24130
  Number of binary variables: 558
  Number of integer variables: 558
  Number of continuous variables: 23572
  Number of nonzeros: 212698
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 1.0112519264221191
  Error rc: 0
  Time: 1.2010555267333984
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

